In [ ]:
#Demo: http://weatherforecast.somee.com/
# This is the method code for C#, ASP.NET MVC
# Controller: DataTrainingController.cs

using System;
using System.Collections.Generic;
using System.Linq;
using System.Web;
using System.Web.Mvc;
using Model;
using Model.EF;
using Model.DAO;
using System.Data.SqlClient;
using System.Web.Configuration;

namespace WeatherForecasting.Controllers
{
    public class DataTrainingController : Controller
    {
        private DataTrainingDbContext db = new DataTrainingDbContext();
        // GET: DataTraining
        [OutputCache(Duration = 2592000, Location = System.Web.UI.OutputCacheLocation.Server)]
        public ActionResult Index(string searchString, int page = 1, int pageSize = 10)
        {
            var dao = new DataTrainingDao();
            var model = dao.ListAllPaging(searchString, page, pageSize);
            ViewBag.SearchString = searchString;
            ViewBag.STT = page * 10 - 9;
            return View(model);
        }

        public ActionResult Prediction()
        {
            return View();
        }
        [HttpPost]
        public ActionResult Prediction(Model.EF.DataTraining item)
        {
            if (ModelState.IsValid)
            {
                string result = "";
                decimal P_yes = Pnew("yes", item.outlook, item.temp, item.humidity, item.windy);
                decimal P_no = Pnew("no", item.outlook, item.temp, item.humidity, item.windy);
                ViewBag.Outlook = item.outlook;
                ViewBag.Temp = item.temp;
                ViewBag.Humidity = item.humidity;
                ViewBag.Windy = item.windy;
                if (P_yes > P_no)
                {
                    result = "yes";
                    ViewBag.Result = result;
                }
                else
                {
                    result = "no";
                    ViewBag.Result = result;
                }    
            }
            return View(item);
        }
        public ActionResult PredictionResult(string result)
        {
            ViewBag.Result = result;
            return View();
        }
        public decimal Pnew(string play, string outlook, string temp, string humidity, string windy)
        {
            decimal P_outlook = P_feature("outlook", outlook, play);
            decimal P_temp = P_feature("temp", temp, play);
            decimal P_humidity = P_feature("humidity", humidity, play);
            decimal P_windy = P_feature("windy", windy, play);
            decimal P_play = P(play);
            decimal result = P_outlook * P_temp * P_humidity * P_windy * P_play;
            return result;
        }
        public decimal P_feature(string feature, string value, string play)
        {
            decimal result = 0;
            //Tử số: {Số buổi feature=value và Play=yes/no} + 1
            decimal numerator = NumberOf(feature, value, play) + 1;
            //Mẫu số:{Số buổi Play=yes/no}+𝑆ố 𝑔𝑖á 𝑡𝑟ị 𝑟ờ𝑖 𝑟ạ𝑐 𝑐ủ𝑎 𝑜𝑢𝑡𝑙𝑜𝑜k
            decimal denominator = NumberPlay(play) + NumberDiscreteValueOf(feature);
            if (denominator == 0) 
                return result;
            else result = Math.Round(numerator / denominator, 2);

            return result;
        }
        public decimal P(string play)
        {
            decimal result = 0;
            //Tử số: {Số buổi Play=No} + 1
            decimal numerator = NumberPlay(play) + 1;
            //Mẫu số:({Tổng số buổi} + Số giá trị rời rạc của Play)
            decimal denominator = TotalTraining() + NumberDiscreteValueOf(play);
            if (denominator == 0)
                return result;
            else
                result = Math.Round(numerator / denominator, 2);
            return result;
        }
        public decimal NumberOf(string feature, string value, string play)
        {
            int result = 0;
            switch (feature)
            {
                case "outlook":
                    result = db.DataTrainings.Where(x => x.outlook == value && x.play == play).Count();
                    break;
                case "temp":
                    result = db.DataTrainings.Where(x => x.temp == value && x.play == play).Count();
                    break;
                case "humidity":
                    result = db.DataTrainings.Where(x => x.humidity == value && x.play == play).Count();
                    break;
                case "windy":
                    result = db.DataTrainings.Where(x => x.windy == value && x.play == play).Count();
                    break;
            }
            return result;
        }
        public decimal NumberPlay(string play)
        {
            int result = 0;
            result = db.DataTrainings.Where(x => x.play == play).Count();
            return result;
        }
        public decimal NumberDiscreteValueOf(string value)
        {
            decimal result = 0;
            switch (value)
            {
                case "outlook":
                    result = db.DataTrainings.Select(x => x.outlook).Distinct().Count();
                    break;
                case "temp":
                    result = db.DataTrainings.Select(x => x.temp).Distinct().Count();
                    break;
                case "humidity":
                    result = db.DataTrainings.Select(x => x.humidity).Distinct().Count();
                    break;
                case "windy":
                    result = db.DataTrainings.Select(x => x.windy).Distinct().Count();
                    break;
                case "play":
                    result = db.DataTrainings.Select(x => x.play).Distinct().Count();
                    break;
                default:
                    break;
            }
            return result;
        }
        public decimal TotalTraining()
        {
            decimal result = db.DataTrainings.ToList().Count();
            return result;
        }

        // GET: DataTraining/Create
        public ActionResult Create()
        {
            return View();
        }

        // POST: DataTraining/Create
        [HttpPost]
        public ActionResult Create(Model.EF.DataTraining item)
        {
            if (ModelState.IsValid)
            {
                var dao = new DataTrainingDao();
                int id = dao.Insert(item);
                if (id > 0)
                {
                    return RedirectToAction("Index", "DataTraining");
                }
                else
                {
                    ModelState.AddModelError("", "Thêm dữ liệu không thành công");
                }
            }
            return View("Index");
        }

        // GET: DataTraining/Details/5
        public ActionResult Details(int id)
        {
            return View();
        }
        // GET: DataTraining/Edit/5
        public ActionResult Edit(int id)
        {
            return View();
        }

        // POST: DataTraining/Edit/5
        [HttpPost]
        public ActionResult Edit(int id, FormCollection collection)
        {
            try
            {
                // TODO: Add update logic here

                return RedirectToAction("Index");
            }
            catch
            {
                return View();
            }
        }

        // GET: DataTraining/Delete/5
        public ActionResult Delete(int id)
        {
            return View();
        }

        // POST: DataTraining/Delete/5
        [HttpPost]
        public ActionResult Delete(int id, FormCollection collection)
        {
            try
            {
                // TODO: Add delete logic here

                return RedirectToAction("Index");
            }
            catch
            {
                return View();
            }
        }
    }
}


In [ ]:
#View: Prediction.cshtml

@model Model.EF.DataTraining
@{
    ViewBag.Title = "Prediction";
    Layout = "~/Views/Shared/_Layout.cshtml";
}
<div class="form-group">
    @if (ViewBag.Result == "yes")
    {
        <h2 style="color:#007bff; text-align: center"><img src="~/Content/rain.jpg" width="10%" /> Take an umbrella! It will rain today!</h2>
    }
    else if (ViewBag.Result == "no")
    {
        <h2 style="color:rgb(244,169,58); text-align: center"><img src="~/Content/sun.png" width="7%" /> Don't Worry! <span style="color:rgb(101,159,19);">Let's enjoy!</span></h2>
    }
    else
    {
        <h2 style="color:rgb(101,159,19); text-align: center;"><img src="~/Content/typing_blue_background.png" style="padding-top: -5px" width="7%" /> Working...</h2>
    }
</div>
<h1 style="text-align: center; font-weight: 700">Prediction weather</h1>

@using (Html.BeginForm("Prediction", "DataTraining", FormMethod.Post, new { enctype = "multipart/form-data", @class = "uk-form-stacked" }))
{
    @Html.AntiForgeryToken()

    <div class="form-horizontal">
        @if (ViewBag.Outlook != null)
        {
            <h4 style="text-align: center">Outlook: @ViewBag.Outlook - Temp: @ViewBag.Temp - Humidity: @ViewBag.Humidity - Windy: @ViewBag.Windy</h4>
        }
        else
        {
        <h4 style="text-align: center">How about your outlook, temp, humidity, windy today? Please fill in to predict!</h4>
        }
        <hr />
         <select class="form-control center-block" id="outlook" name="outlook">
            <option>Select Outlook</option>
            <option>sunny</option>
            <option>overcast</option>
            <option>rainy</option>
        </select>
        <br />
        <select class="form-control center-block" id="temp" name="temp">
            <option>Select Temp</option>
            <option>hot</option>
            <option>mild</option>
            <option>cold</option>
        </select>
        <br />
         <select class="form-control center-block" id="humidity" name="humidity">
            <option>Select Humidity</option>
            <option>high</option>
            <option>normal</option>
        </select>
        <br />
        <select class="form-control center-block" id="windy" name="windy">
            <option>Select Windy</option>
            <option>false</option>
            <option>true</option>
        </select>
        <br />
        <input type="submit" value="Predict" class="btn btn-primary center-block" />
    </div>
}
<br />
<div>
    <p style="text-align: center"><a href="/DataTraining/Prediction">Continue with another</a></p>
</div>

@section Scripts {
    @Scripts.Render("~/bundles/jqueryval")
}




In [ ]:
#View: Index.cshtml (for displaying list of data-training) (not recommend)

@using PagedList.Mvc;
@model  PagedList.IPagedList<Model.EF.DataTraining>
@{
    ViewBag.Title = "Index";
    Layout = "~/Views/Shared/_Layout.cshtml";
}
<h2>Our data-training we have now</h2>
@*<p>
    @Html.ActionLink("Create New", "Create")
</p>*@
@using (Html.BeginForm("Index", "DataTraining"))
{
    <label>Search:</label>
    <input type="text" id="searchString" name="searchString" />
    <button type="submit">Search</button>
}
<br />

<table class="table">
    <tr>
        <th>id</th>
        <th>Outlook</th>
        <th>Temp</th>
        <th>Humidity</th>
        <th>Windy</th>
        <th>Play</th>
    </tr>
    @{ int i = ViewBag.STT;}
    @foreach (var item in Model)
    {
        <tr id="@item.id">
            <td>@i</td>
            <td>
                @item.outlook
            </td>
            <td>@item.temp</td>
            <td>@item.humidity</td>
            <td>@item.windy</td>
            <td>@item.play</td>
        </tr>
        i++;
    }
</table>

@Html.PagedListPager(Model, page => Url.Action("Index", new { page, @searchString = ViewBag.SearchString }))


In [ ]:
# Model Layer - DAO: DataTrainingDao.cs 
#Focusing on ListAllPaging method, the rest of methods you can ignore or not (up to you)

using System;
using System.Collections.Generic;
using System.Linq;
using Model.EF;
using PagedList;

namespace Model.DAO
{
    public class DataTrainingDao
    {
        DataTrainingDbContext db = null;
        public DataTrainingDao()
        {
            db = new DataTrainingDbContext();
        }
        public DataTraining GetByID(long id)
        {
            return db.DataTrainings.Find(id);
        }
        public int Insert(DataTraining entity)
        {
            //Tạo mới tham số đối tượng: entity 
            db.DataTrainings.Add(entity);
            db.SaveChanges();
            return entity.id;
        }
        public DataTraining ViewDetail(int id)
        {
            return db.DataTrainings.Find(id);
        }

        public bool Delete(int id)
        {
            try
            {
                var DataTraining = db.DataTrainings.Find(id);
                db.DataTrainings.Remove(DataTraining);
                db.SaveChanges();
                return true;
            }
            catch (Exception)
            {
                return false;
                //throw;
            }
        }
        public IEnumerable<DataTraining> ListAllPaging(string searchString, int page, int pageSize)
        {
            IQueryable<DataTraining> model = db.DataTrainings;
            if (!string.IsNullOrEmpty(searchString))
            {
                model = model.Where(x => x.outlook.Contains(searchString) || x.temp.Contains(searchString) || x.humidity.Contains(searchString) || x.windy.Contains(searchString));
            }
            return model.OrderByDescending(x => x.id).ToPagedList(page, pageSize);
        }
        public List<DataTraining> GetChildren(int count)
        {
            return db.DataTrainings.OrderByDescending(a => a.id).Take(count).ToList();
        }
        public IEnumerable<DataTraining> ListAll()
        {
            return db.DataTrainings.ToList();
        }
    }
}
